In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from IPython.display import display, Image
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping



In [8]:
# Define the path to the dataset folders
happy_folder = "/kaggle/input/pets-facial-expression-dataset/happy"
sad_folder = "/kaggle/input/pets-facial-expression-dataset/Sad"
angry_folder = "/kaggle/input/pets-facial-expression-dataset/Angry"


# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images

# Load images and labels for each emotion
happy_images = load_images_from_folder(happy_folder)
sad_images = load_images_from_folder(sad_folder)
angry_images = load_images_from_folder(angry_folder)


# Create labels for each emotion category
happy_labels = [0] * len(happy_images)
sad_labels = [1] * len(sad_images)
angry_labels = [2] * len(angry_images)


# Concatenate images and labels
X = np.array(happy_images + sad_images + angry_images)
y = np.array(happy_labels + sad_labels + angry_labels)

# Normalize pixel values to range [0, 1]
X = X.astype('float32') / 255.0

# One-hot encode the labels
y = np_utils.to_categorical(y, 3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Training and Evaluation

In [9]:
# Build the CNN model
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

print(model.summary())
# Compile the model with class weights
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_130 (Conv2D)         (None, 46, 46, 64)        640       
                                                                 
 batch_normalization_3 (Batc  (None, 46, 46, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_131 (Conv2D)         (None, 44, 44, 64)        36928     
                                                                 
 max_pooling2d_65 (MaxPoolin  (None, 22, 22, 64)       0         
 g2D)                                                            
                                                                 
 dropout_97 (Dropout)        (None, 22, 22, 64)        0         
                                                                 
 conv2d_132 (Conv2D)         (None, 20, 20, 128)     

In [10]:

# Calculate class weights
total_samples = len(y_train)
class_weights = {0: total_samples / np.sum(y_train[:, 0]), 
                 1: total_samples / np.sum(y_train[:, 1]), 
                 2: total_samples / np.sum(y_train[:, 2])}
# Train the model with class weights
history = model.fit(X_train.reshape(-1, 48, 48, 1), y_train, batch_size=32, epochs=100, validation_split=0.1, class_weight=class_weights,verbose=0)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test.reshape(-1, 48, 48, 1), y_test)
losstr, accuracytr = model.evaluate(X_train.reshape(-1, 48, 48, 1), y_train)


# Save the trained model
model.save("facial_expression_model_v2.0.h5")


2024-06-11 18:35:20.703445: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_32/dropout_97/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


19/19 [==============================] - 0s 5ms/step - loss: 4.5114 - accuracy: 0.6200


In [11]:
print(f"Test accuracy: {accuracy*100:.2f}%")
print(f"Train accuracy: {accuracytr*100:.2f}%")

Test accuracy: 46.67%
Train accuracy: 62.00%


## Hyperparamter tuning

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

# Function to create and compile the CNN model
def create_cnn_model(conv_filters_1=64, conv_filters_2=128, dense_units=512, dropout_rate=0.5):
    model = Sequential()
    model.add(Conv2D(conv_filters_1, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(Conv2D(conv_filters_1, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(conv_filters_2, (3, 3), activation='relu'))
    model.add(Conv2D(conv_filters_2, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))

    # Compile the model with categorical_crossentropy loss
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier wrapper for the CNN model
cnn_model = KerasClassifier(build_fn=create_cnn_model, epochs=10, batch_size=64, verbose=1)

# Define the hyperparameter space to search
param_grid = {
    'conv_filters_1': [32, 64, 128],
    'conv_filters_2': [64, 128, 256],
    'dense_units': [256, 512, 1024],
    'dropout_rate': [0.25, 0.5, 0.75]
}

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(cnn_model, param_distributions=param_grid, n_iter=10, cv=3, verbose=1, random_state=42)

# Fit the RandomizedSearchCV to the data
random_search_result = random_search.fit(X_train.reshape(-1, 48, 48, 1), y_train)

# Print the best hyperparameters found
print("Best Parameters:", random_search_result.best_params_)

# Retrieve the best model
best_model = random_search_result.best_estimator_

# Evaluate the best model on test data
loss, accuracy = best_model.model.evaluate(X_test.reshape(-1, 48, 48, 1), y_test)
print(f"Test accuracy of the best model: {accuracy*100:.2f}%")


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/10


/tmp/ipykernel_32/2133966319.py:27: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  cnn_model = KerasClassifier(build_fn=create_cnn_model, epochs=10, batch_size=64, verbose=1)
2024-06-11 18:35:49.275737: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_33/dropout_101/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


7/7 [==============================] - 2s 12ms/step - loss: 1.1507 - accuracy: 0.3425
Epoch 2/10
7/7 [==============================] - 0s 11ms/step - loss: 1.4877 - accuracy: 0.3525
Epoch 3/10
7/7 [==============================] - 0s 11ms/step - loss: 1.1411 - accuracy: 0.3600
Epoch 4/10
7/7 [==============================] - 0s 11ms/step - loss: 2.2924 - accuracy: 0.3875
Epoch 5/10
7/7 [==============================] - 0s 11ms/step - loss: 2.2882 - accuracy: 0.3800
Epoch 6/10
7/7 [==============================] - 0s 11ms/step - loss: 1.3007 - accuracy: 0.3925
Epoch 7/10
7/7 [==============================] - 0s 11ms/step - loss: 2.5027 - accuracy: 0.3825
Epoch 8/10
7/7 [==============================] - 0s 11ms/step - loss: 3.9653 - accuracy: 0.4300
Epoch 9/10
7/7 [==============================] - 0s 11ms/step - loss: 1.9874 - accuracy: 0.3875
Epoch 10/10
4/4 [==============================] - 0s 4ms/step - loss: 1.4847 - accuracy: 0.4150
Epoch 1/10


2024-06-11 18:35:52.577859: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_34/dropout_104/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


7/7 [==============================] - 2s 11ms/step - loss: 1.1481 - accuracy: 0.3225
Epoch 2/10
7/7 [==============================] - 0s 11ms/step - loss: 1.1959 - accuracy: 0.3475
Epoch 3/10
7/7 [==============================] - 0s 11ms/step - loss: 1.2590 - accuracy: 0.3775
Epoch 4/10
7/7 [==============================] - 0s 11ms/step - loss: 1.4573 - accuracy: 0.3550
Epoch 5/10
7/7 [==============================] - 0s 11ms/step - loss: 3.4259 - accuracy: 0.3775
Epoch 6/10
7/7 [==============================] - 0s 11ms/step - loss: 2.2240 - accuracy: 0.3875
Epoch 7/10
7/7 [==============================] - 0s 11ms/step - loss: 2.0601 - accuracy: 0.3850
Epoch 8/10
7/7 [==============================] - 0s 11ms/step - loss: 2.2590 - accuracy: 0.3600
Epoch 9/10
7/7 [==============================] - 0s 11ms/step - loss: 1.9712 - accuracy: 0.3575
Epoch 10/10
4/4 [==============================] - 0s 4ms/step - loss: 1.3236 - accuracy: 0.3500
Epoch 1/10


...

7/7 [==============================] - 2s 10ms/step - loss: 1.2279 - accuracy: 0.3200
Epoch 2/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0999 - accuracy: 0.3375
Epoch 3/10
7/7 [==============================] - 0s 9ms/step - loss: 1.1077 - accuracy: 0.3075
Epoch 4/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0967 - accuracy: 0.3450
Epoch 5/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0890 - accuracy: 0.3750
Epoch 6/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0673 - accuracy: 0.4575
Epoch 7/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0807 - accuracy: 0.4150
Epoch 8/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0740 - accuracy: 0.4075
Epoch 9/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0734 - accuracy: 0.4500
Epoch 10/10
4/4 [==============================] - 0s 5ms/step - loss: 1.0632 - accuracy: 0.4500
Epoch 1/10


2024-06-11 18:37:31.072916: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_62/dropout_188/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


7/7 [==============================] - 2s 10ms/step - loss: 1.1460 - accuracy: 0.3450
Epoch 2/10
7/7 [==============================] - 0s 10ms/step - loss: 1.1770 - accuracy: 0.3800
Epoch 3/10
7/7 [==============================] - 0s 9ms/step - loss: 1.0884 - accuracy: 0.4150
Epoch 4/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0648 - accuracy: 0.4775
Epoch 5/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0954 - accuracy: 0.4600
Epoch 6/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0933 - accuracy: 0.3700
Epoch 7/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0676 - accuracy: 0.5200
Epoch 8/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0553 - accuracy: 0.4825
Epoch 9/10
7/7 [==============================] - 0s 10ms/step - loss: 1.0494 - accuracy: 0.4800
Epoch 10/10
4/4 [==============================] - 0s 5ms/step - loss: 1.1115 - accuracy: 0.4200
Epoch 1/10


2024-06-11 18:37:34.443658: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_63/dropout_191/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


10/10 [==============================] - 3s 33ms/step - loss: 1.1476 - accuracy: 0.3333
Epoch 2/10
10/10 [==============================] - 0s 14ms/step - loss: 1.0988 - accuracy: 0.3750
Epoch 3/10
10/10 [==============================] - 0s 14ms/step - loss: 1.0948 - accuracy: 0.3733
Epoch 4/10
10/10 [==============================] - 0s 14ms/step - loss: 1.0764 - accuracy: 0.4550
Epoch 5/10
10/10 [==============================] - 0s 14ms/step - loss: 1.0468 - accuracy: 0.4683
Epoch 6/10
10/10 [==============================] - 0s 14ms/step - loss: 1.0207 - accuracy: 0.5033
Epoch 7/10
10/10 [==============================] - 0s 14ms/step - loss: 1.0145 - accuracy: 0.5000
Epoch 8/10
10/10 [==============================] - 0s 14ms/step - loss: 0.9830 - accuracy: 0.5633
Epoch 9/10
10/10 [==============================] - 0s 14ms/step - loss: 0.9649 - accuracy: 0.5550
Epoch 10/10
10/10 [==============================] - 0s 14ms/step - loss: 0.9109 - accuracy: 0.6067
Best Parameters: {'d